In [1]:
#imported librarys 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from keras.utils import to_categorical
import matplotlib.pyplot as plt 
import keras
from time import time
from keras.layers.core import Activation, Dropout, Dense, Masking
#padding and tokenization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load the dataset
dataset1_path = '/content/drive/MyDrive/Colab Notebooks/players_preprocessed_new_data (1).csv'
dataset2_path = '/content/drive/MyDrive/Colab Notebooks/preprocessed_airlines_data (1) (1).csv'

tweet_airline_sentiment = pd.read_csv(dataset2_path)
tweet_player_sentiment = pd.read_csv(dataset1_path)
tweet_airline_sentiment

,Unnamed: 0,text,airline_sentiment
0,0,said,neutral
1,1,plus youve added commercial experience tacky,positive
2,2,didnt today must mean need take another trip,neutral
3,3,really aggressive blast obnoxious entertainmen...,negative
4,4,really big bad thing,negative
...,...,...,...
14635,14635,thank got different flight chicago,positive
14636,14636,leaving 20 minute late flight warning communic...,negative
14637,14637,please bring american airline,neutral
14638,14638,money change flight dont answer phone suggesti...,negative


In [4]:
# Change the column names
new_column_names = ['Unnamed: 0','text', 'airline_sentiment']
tweet_player_sentiment.columns = new_column_names

# Append dataset 10-14 to dataset 1-10
sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)

# Optional: Reset the index of the merged dataset
sentiment_data = sentiment_data.reset_index(drop=True)

# Rename a single column
sentiment_data.rename(columns={'airline_sentiment': 'sentiment'}, inplace=True)

sentiment_data = sentiment_data.drop('Unnamed: 0', axis=1)

# Save the merged dataset to a new CSV file
sentiment_data.to_csv('Sentiment_dataset.csv', index=False)


<ipython-input-4-c39e254d91d7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)


In [5]:
sentiment_data

,text,sentiment
0,said,neutral
1,plus youve added commercial experience tacky,positive
2,didnt today must mean need take another trip,neutral
3,really aggressive blast obnoxious entertainmen...,negative
4,really big bad thing,negative
...,...,...
333925,trophy best playstyle serie problem,positive
333926,excited,positive
333927,sleep well dont worry cuz ur team fucking shit,negative
333928,google daniel twitch franco jayz,neutral


In [6]:
# Encode the sentiment labels
label_encoder = LabelEncoder()
sentiment_data['sentiment'] = label_encoder.fit_transform(sentiment_data['sentiment'])

In [7]:
# train, val, test split

x_train, xtest, y_train, ytest = train_test_split(sentiment_data.text.values,sentiment_data.sentiment,stratify=sentiment_data.sentiment.values, test_size=0.3,random_state=1)
y_train = to_categorical(y_train)
x_val = xtest[0:25000]
y_val = to_categorical(ytest[0:25000])
x_test = xtest[25000:]
y_test = ytest[25000:]

# Convert non-string values to strings
sentiment_data.text = sentiment_data.text.astype(str)
x_train = x_train.astype(str)
x_val = x_val.astype(str)
x_test = x_test.astype(str)

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentiment_data.text.values)


X_train = tokenizer.texts_to_sequences(x_train)
X_val = tokenizer.texts_to_sequences(x_val)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100
#padding
X_train = pad_sequences(X_train, padding='pre', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='pre', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='pre', maxlen=maxlen)

word_index = tokenizer.word_index

In [8]:
#import glove embeddings

embeddings_index = {}
f = open(os.path.join( '/content/drive/MyDrive/Colab Notebooks/word_embeddings.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [9]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU available')
else:
    print('GPU not available')

# Set TensorFlow to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


GPU available


In [10]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Prepare your data and labels
embedding_dim = 100
num_classes = 3

# Define the model architecture
model = Sequential()

model.add(Embedding(vocab_size,
                    embedding_dim, 
                    input_length=maxlen))
model.add(layers.LSTM(64, 
                      return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64,
                      return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, 
                      return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='sigmoid'))



In [11]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model/', save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=BinaryCrossentropy(),
              metrics=['accuracy', AUC(name='auc')])

In [12]:
sentiment_data['sentiment']

0         1
1         2
2         1
3         0
4         0
         ..
333925    2
333926    2
333927    0
333928    1
333929    0
Name: sentiment, Length: 333930, dtype: int64

In [13]:
frequencies = pd.value_counts(sentiment_data['sentiment'])

frequencies

0    119036
2    111705
1    103189
Name: sentiment, dtype: int64

In [14]:
weights = {0: frequencies.sum() / frequencies[0], 1: frequencies.sum() / frequencies[1],2: frequencies.sum() / frequencies[2]}
weights

{0: 2.8052857958936794, 1: 3.2361007471726637, 2: 2.9893917013562508}

In [15]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[cp], class_weight=weights)

Epoch 1/20
7304/7305 [============================>.] - ETA: 0s - loss: 1.2780 - accuracy: 0.6850 - auc: 0.8631

7305/7305 [==============================] - 191s 24ms/step - loss: 1.2779 - accuracy: 0.6850 - auc: 0.8631 - val_loss: 0.3339 - val_accuracy: 0.7788 - val_auc: 0.9202
Epoch 2/20
7304/7305 [============================>.] - ETA: 0s - loss: 0.9851 - accuracy: 0.7908 - auc: 0.9246

7305/7305 [==============================] - 143s 20ms/step - loss: 0.9851 - accuracy: 0.7908 - auc: 0.9246 - val_loss: 0.3204 - val_accuracy: 0.7896 - val_auc: 0.9267
Epoch 3/20
7304/7305 [============================>.] - ETA: 0s - loss: 0.9454 - accuracy: 0.7994 - auc: 0.9308

7305/7305 [==============================] - 140s 19ms/step - loss: 0.9454 - accuracy: 0.7994 - auc: 0.9308 - val_loss: 0.3158 - val_accuracy: 0.7926 - val_auc: 0.9290
Epoch 4/20
7303/7305 [============================>.] - ETA: 0s - loss: 0.9264 - accuracy: 0.8046 - auc: 0.9337

7305/7305 [==============================] - 140s 19ms/step - loss: 0.9264 - accuracy: 0.8046 - auc: 0.9337 - val_loss: 0.3126 - val_accuracy: 0.7943 - val_auc: 0.9305
Epoch 5/20
7305/7305 [==============================] - 127s 17ms/step - loss: 0.9134 - accuracy: 0.8067 - auc: 0.9356 - val_loss: 0.3130 - val_accuracy: 0.7972 - val_auc: 0.9306
Epoch 6/20
7304/7305 [============================>.] - ETA: 0s - loss: 0.8998 - accuracy: 0.8096 - auc: 0.9375

7305/7305 [==============================] - 139s 19ms/step - loss: 0.8999 - accuracy: 0.8096 - auc: 0.9375 - val_loss: 0.3116 - val_accuracy: 0.7960 - val_auc: 0.9312
Epoch 7/20
7305/7305 [==============================] - 127s 17ms/step - loss: 0.8873 - accuracy: 0.8120 - auc: 0.9394 - val_loss: 0.3145 - val_accuracy: 0.7928 - val_auc: 0.9297
Epoch 8/20
7305/7305 [==============================] - 127s 17ms/step - loss: 0.8753 - accuracy: 0.8150 - auc: 0.9410 - val_loss: 0.3152 - val_accuracy: 0.7931 - val_auc: 0.9296
Epoch 9/20
7305/7305 [==============================] - 126s 17ms/step - loss: 0.8631 - accuracy: 0.8173 - auc: 0.9427 - val_loss: 0.3180 - val_accuracy: 0.7947 - val_auc: 0.9289
Epoch 10/20
7305/7305 [==============================] - 126s 17ms/step - loss: 0.8508 - accuracy: 0.8204 - auc: 0.9443 - val_loss: 0.3210 - val_accuracy: 0.7936 - val_auc: 0.9278
Epoch 11/20
7305/7305 [==============================] - 126s 17ms/step - loss: 0.8412 - accuracy: 0.8224 - auc: 0.

In [16]:
from tensorflow.keras.models import load_model

best_model = load_model('model/')

In [17]:
# Save the trained model
model.save("rnn_lstm_new_model.h5")

# Save the model weights only
model.save_weights("model_rnn_lstm_new_weights.h5")

In [18]:
test_predictions = (best_model.predict(X_test) > 0.5).astype(int)

from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Convert target labels to one-hot encoding
y_test_encoded = to_categorical(y_test, num_classes=3)

print(classification_report(y_test_encoded, test_predictions))

2350/2350 [==============================] - 23s 8ms/step
              precision    recall  f1-score   support

           0       0.84      0.83      0.84     26805
           1       0.76      0.68      0.72     23313
           2       0.84      0.81      0.82     25061

   micro avg       0.82      0.78      0.80     75179
   macro avg       0.82      0.77      0.79     75179
weighted avg       0.82      0.78      0.80     75179
 samples avg       0.78      0.78      0.78     75179



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
